#### 1. Instalação de Bibliotecas

In [0]:
pip install translate

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install unidecode

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install schedule

Python interpreter will be restarted.
Python interpreter will be restarted.


#### 2. Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.utils import AnalysisException
from translate import Translator
import re
from unidecode import unidecode
import schedule
import time
from datetime import datetime

#### 3. Constantes

In [0]:
translator = Translator(to_lang="pt", from_lang='en')

path_raw = '/Filestore/projeto/dados_raw'
path_dados_completos = '/Filestore/projeto/dados_final/dados_completos'
path_quantidade_noticias_dia = '/Filestore/projeto/dados_final/quantidade_noticias_dia'
path_quantidade_noticias_mes = '/Filestore/projeto/dados_final/quantidade_noticias_mes'
path_quantidade_noticias_ano = '/Filestore/projeto/dados_final/quantidade_noticias_ano'
path_quantidade_noticias_fonte = '/Filestore/projeto/dados_final/quantidade_noticias_fonte'
path_quantidade_noticias_autor = '/Filestore/projeto/dados_final/quantidade_noticias_autor'
path_quantidade_palavras_chaves = '/Filestore/projeto/dados_final/quantidade_palavras_chaves'

palavras_chave =  ['outubro rosa', 'DNA', 'doença genética', 'terapia genética']

#### 4. Leitura dos dados raw

In [0]:
def read_parquet(path):
    return spark.read.parquet(path)

display(read_parquet(path_raw))

url,source,author,title,description,urlToImage,publishedAt,content,data_processamento
http://deadline.com/2024/10/the-hill-duo-film-tv-company-enliven-content-race-horse-drama-codys-wish-1236108600/,"List(null, Deadline)",Andreas Wiseman,‘The Hill’ Duo Launch Film & TV Company Enliven Content & Set True Race Horse Story ‘Cody’s Wish’ As First Project,"EXCLUSIVE: Director Jeff Celentano and producer Warren Ostergard, who recently collaborated on Dennis Quaid movie The Hill, have reunited to launch Enliven Content, a film and TV production company with a focus on “true, inspiring, and uplifting stories”. The…",https://deadline.com/wp-content/uploads/2024/10/The-Hill-2.webp?w=1024,2024-10-07T14:40:00Z,"EXCLUSIVE: Director Jeff Celentano and producer Warren Ostergard, who recently collaborated on Dennis Quaid movie The Hill, have reunited to launch Enliven Content, a film and TV production company w… [+3040 chars]",2024-10-20T17:24:11.578+0000
http://twistedsifter.com/2024/10/is-the-family-that-walks-on-all-fours-proof-of-backwards-evolution/,"List(null, Twistedsifter.com)",Trisha Leigh,Is The Family That Walks On All Fours Proof Of “Backwards Evolution?”,"In the end, it's more of a sad story than an incredible one.",https://twistedsifter.com/wp-content/uploads/2024/09/QuadrupedalFamily.png,2024-10-11T13:21:23Z,Just when we think we know all there is to know about humanity and evolution – namely that it flows in one direction – a family comes along to challenge everything. In this case it’s the Ulas family… [+4779 chars],2024-10-20T17:24:11.578+0000
http://www.vulture.com/article/greys-anatomy-recap-season-21-episode-2-take-me-to-church.html,"List(null, Vulture)",Laura Bradley,Grey’s Anatomy Recap: Don’t Lift My Albatross,"Everyone’s going through it this week on Grey’s, and too many of them are determined to do it alone.",https://pyxis.nymag.com/v1/imgs/812/09c/8b0b0d1714953f82e27b9dd8615e5e3625-173882-0127.1x.rsocial.w1200.jpg,2024-10-04T13:43:34Z,"What is it about surgeons on Greys Anatomy and their super-secret tumors? From Amelia Shepherd to Nicole Herman, from Izzie Stevens to Richard Webber, these doctors simply love to hide life-threateni… [+7848 chars]",2024-10-20T17:24:11.578+0000
https://abcnews.go.com/GMA/Family/father-daughter-sue-after-alleged-ivf-mix-show/story?id=114425359,"List(abc-news, ABC News)",Yi-Jin Yu,"Father, daughter sue after alleged IVF mix-up show there is no biological relation",A Las Vegas father and daughter have filed a lawsuit against an IVF doctor and related staff after the results of a DNA test allegedly showed there was no relation.,https://i.abcnewsfe.com/a/ea172e2e-6f17-4885-a5f7-34414e21ef25/ancestry-dna-gty-thg-241002_1727878148315_hpMain_16x9.jpg?w=1600,2024-10-03T18:25:24Z,"A Las Vegas father and daughter have filed a lawsuit against an IVF doctor, embryologist and related staff after the results of a 2023 DNA test allegedly showed the father and daughter, now 18, are n… [+2477 chars]",2024-10-20T17:24:11.578+0000
https://abcnews.go.com/International/wireStory/suspect-1977-melbourne-cold-case-arrested-italy-113898622,"List(abc-news, ABC News)",The Associated Press,Suspect in 1977 Melbourne cold case arrested in Italy,He is suspected of the brutal killing of two women in Australia.,https://s.abcnews.com/images/US/abc_news_default_2000x2000_update_16x9_992.jpg,2024-09-21T18:28:25Z,"ROME -- A man suspected of the brutal killing of two women in Australia nearly a half-century ago has been arrested in Rome on an international arrest warrant, Italian news agency ANSA reported on Sa… [+1210 chars]",2024-10-20T17:24:11.578+0000
https://abcnews.go.com/Politics/prosecutors-request-indefinite-delay-trial-trump-assassination-attempt/story?id=114441982,"List(abc-news, ABC News)",Alexander Mallin,Prosecutors request indefinite delay in trial for Trump assassination attempt suspect,Federal prosecutors requested an indefinite delay in scheduling the trial for the man charged in an apparent assassination

#### 5. Persistência dos dados

In [0]:
def write_parquet(dataframe, path, mode):
  dataframe.write.format('delta').mode(mode).parquet(path)
  print(f"{dataframe.count()} novas notícias foram persistidas em: {path}")

def obter_dados_nao_salvos(dataframe, path):
  try:
    df_leitura = spark.read.parquet(path)
    df_left_join = dataframe.alias("df1").join(df_leitura.alias("df2"), on="url", how="left")
    left_exclude = df_left_join.filter(col("df2.url").isNull()).select("df1.*")
    return left_exclude
  except AnalysisException:
    return dataframe

#### 6. Limpeza dos dados

In [0]:
def limpeza_dados(dataframe):
    # Remoção da coluna 'id' da coluna 'source'
    df = dataframe.withColumn("source", col("source").getItem("name"))
    
    # Remoção de conteúdos 'Removed'
    df = df.filter(col('url') != 'https://removed.com')
    
    # Transformação da coluna publishedAt para TimestampType
    df = df.withColumn("publishedAt", to_timestamp(col("publishedAt"), "yyyy-MM-dd'T'HH:mm:ss'Z'"))

    # Transformação dos nomes das colunas: tradução e snake_case
    translated_columns = list(map(lambda coluna: translator.translate(re.sub(r'([a-z])([A-Z])', r'\1 \2', coluna)).replace(' ', "_").lower(), df.columns))
    
    for original, translated in zip(df.columns, translated_columns):
        df = df.withColumnRenamed(original, unidecode(translated))
    df = df.withColumnRenamed('title', 'titulo')

    # Troca de null por 'desconhecido':
    df = df.na.fill("Desconhecido")

    return df

#### 7. Transformação dos dados

##### 7.1 Quantidade de notícias por ano, mês e dia de publicação

In [0]:
def group_by_year(dataframe, ascending = True):
    df = dataframe.withColumn("ano", year(col("publicado_em:")))
    return df.groupBy("ano").count().orderBy("count", ascending=ascending)

def group_by_month(dataframe, ascending = True):
    df = dataframe.withColumn("mes", date_format(col("publicado_em:"), "MM-yyyy"))
    return df.groupBy("mes").count().orderBy("count", ascending=ascending)

def group_by_day(dataframe, ascending = True):
    df = dataframe.withColumn("data", to_date(col('publicado_em:')))
    return df.groupBy("data").count().orderBy("count", ascending=ascending)


##### 7.2 Quantidade de notícias por fonte e autor

In [0]:
def group_by_source(dataframe, ascending=True):
    return dataframe.groupBy("fonte").count().orderBy("count", ascending=ascending)

def group_by_author(dataframe, ascending=True):
    return dataframe.groupBy("autor").count().orderBy("count", ascending=ascending)

##### 7.3 Quantidade de aparições de 3 palavras chaves por ano, mês e dia de publicação

In [0]:
def group_by_keyword(dataframe, ascending=True):

    df = dataframe.withColumn("palavra_chave",
        when(lower(col("titulo")).contains("outubro rosa"), lit("outubro rosa"))
        .when(lower(col("titulo")).contains("dna"), lit("DNA"))
        .when(lower(col("titulo")).contains("doença genética"), lit("doença genética"))
        .when(lower(col("titulo")).contains("terapia genética"), lit("terapia genética")))

    df = df.filter(df["palavra_chave"].isNotNull())
    df = df.withColumn("data", to_date(col('publicado_em:')))

    df = df.groupBy("palavra_chave", "data").count().orderBy("data", ascending=ascending)

    return df



#### 8. Atualização dos dados transformados

In [0]:
def update_transformed_data():
    print(datetime.now())
    df_raw = read_parquet(path_raw)

    #limpeza dos dados:
    dados_limpos = limpeza_dados(df_raw)
    df_unicos = obter_dados_nao_salvos(dados_limpos, path_dados_completos)
    write_parquet(df_unicos, path_dados_completos, "append")

    #limpeza dados temporários a serem tratados
    df_raw = spark.createDataFrame(spark.sparkContext.emptyRDD(), df_raw.schema) 
    write_parquet(df_raw, path_raw, "overwrite")

    df_dados_completos = read_parquet(path_dados_completos)
        
    # #Quantidade de notícias por dia:
    df_group_by_day = group_by_day(df_dados_completos)
    write_parquet(df_group_by_day, path_quantidade_noticias_dia, "overwrite")

    # #Quantidade de notícias por mês:
    df_group_by_month = group_by_month(df_dados_completos)
    write_parquet(df_group_by_month, path_quantidade_noticias_mes, "overwrite")
        
    # #Quantidade de notícias por ano:
    df_group_by_year = group_by_year(df_dados_completos)
    write_parquet(df_group_by_year, path_quantidade_noticias_ano, "overwrite")

    # #Quantidade de notícias por fonte:
    df_group_by_source = group_by_source(df_dados_completos)
    write_parquet(df_group_by_source, path_quantidade_noticias_fonte, "overwrite")

    # #Quantidade de notícias por autor:
    df_group_by_author = group_by_author(df_dados_completos)
    write_parquet(df_group_by_author, path_quantidade_noticias_autor, "overwrite")

    # #Quantidade de notícias por palavras-chave:
    df_group_by_keyword = group_by_keyword(df_dados_completos)
    write_parquet(df_group_by_keyword, path_quantidade_palavras_chaves, "overwrite")



In [0]:
schedule.every().day.at("17:30").do(update_transformed_data)   #Roda todo dia no mesmo horário. Verificar horário!!!
# schedule.every().minute.do(update_transformed_data)          #executa a cada um minuto. Apenas para teste
while True:
    schedule.run_pending()
    time.sleep(1)

2024-10-20 17:30:00.097225
610 novas notícias foram persistidas em: /Filestore/projeto/dados_final/dados_completos
0 novas notícias foram persistidas em: /Filestore/projeto/dados_raw
31 novas notícias foram persistidas em: /Filestore/projeto/dados_final/quantidade_noticias_dia
2 novas notícias foram persistidas em: /Filestore/projeto/dados_final/quantidade_noticias_mes
1 novas notícias foram persistidas em: /Filestore/projeto/dados_final/quantidade_noticias_ano
122 novas notícias foram persistidas em: /Filestore/projeto/dados_final/quantidade_noticias_fonte
303 novas notícias foram persistidas em: /Filestore/projeto/dados_final/quantidade_noticias_autor
34 novas notícias foram persistidas em: /Filestore/projeto/dados_final/quantidade_palavras_chaves
